In [1]:
import pandas as pd

import sqlalchemy as sqla
from sqlalchemy.engine import URL

In [91]:
connection_url = URL.create(
    "mssql+pyodbc",
    username="loginom",
    password="TSynJM8PisaL5Lf8nEbS",
    host="10.101.10.157",
    port=None,
    database="erp_data",
    query={
        "driver": "ODBC Driver 18 for SQL Server",
        "TrustServerCertificate": "yes",
    },
)

engin = sqla.create_engine(connection_url)

query = 'SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED;\
SET NOCOUNT ON;\
Select \
    loginom.ref_sql_to_string(Номенклатура._IDRRef) As Номенклатура_Key, \
    loginom.ref_sql_to_string(Номенклатура._ParentIDRRef) As Родитель_Key \
From \
    _Reference354 Номенклатура \
Where \
    Номенклатура._Folder = 0'

with engin.connect() as conn:
    child_parent = pd.read_sql_query(query, conn)
child_parent = child_parent[child_parent['Родитель_Key'] != '00000000000000000000000000000000']

In [92]:
def find_root(row, parent_col, child_col):
    while row[parent_col] is not None:
        parent_value = row[parent_col]
        if parent_value not in child_parent[child_col].values:
            return parent_value
        row = child_parent[child_parent[child_col] == parent_value].iloc[0]
    return row[parent_col]

In [93]:
child_parent['Корень'] = child_parent.apply(find_root, axis=1, parent_col='Родитель_Key', child_col='Номенклатура_Key')
child_parent

,Номенклатура_Key,Родитель_Key,Корень
0,8120005056955d6411ebdcd9bec7972b,8120005056955d6411ebdcd9b74481a2,80e700505695472911e79deb113ada24
1,8120005056955d6411ebdcdf3a40bed8,8120005056955d6411ebdcd9f0f2dd8b,80e700505695472911e79deb113ada24
2,8120005056955d6411ebdcdfd865b25e,8120005056955d6411ebdcdfd168be48,80e700505695472911e79deb113ada24
3,a29d001a64d3eace11df59b81f820b62,8124005056955d6411ec1536f25413f6,8124005056955d6411ec1536f25413f6
4,8530005056b1f4b511e18e0ce07ae28b,8530005056b1f4b511e18e0ce07ae28a,bef2001731135adc11db68b68a49874c
...,...,...,...
1631,80e500505695472911e77ea05c46ed0a,807b005056b1f4b511e2e21dbfb83ebd,bef2001731135adc11db68b68a49874c
1632,810000505695472911e8689aaa43ef71,807b005056b1f4b511e2e21dbfb83ebd,bef2001731135adc11db68b68a49874c
1633,a46f005056b17b8b11ee2c84ef50c820,807b005056b1f4b511e2e21dbfb83ebd,bef2001731135adc11db68b68a49874c
1634,afd1005056b1f4b511e4bd95fb7c0608,807b005056b1f4b511e2e21dbfb83ebd,bef2001731135adc11db68b68a49874c
